In [ ]:
!pip install -q kaggle

from google.colab import files
files.upload()

!mkdir '/root/.kaggle'
!cp kaggle.json '/root/.kaggle'
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets list

Saving kaggle.json to kaggle.json
ref                                                     title                                          size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------  --------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
nelgiriyewithana/apple-quality                          Apple Quality                                 170KB  2024-01-11 14:31:07          20511        455  1.0              
willianoliveiragibin/alcohol-consumption                Alcohol Consumption                             7KB  2024-02-21 19:49:27           1077         29  1.0              
syedanwarafridi/vehicle-sales-data                      Vehicle Sales Data                             19MB  2024-02-21 20:16:17           1308         28  1.0              
mikhail1681/walmart-sales                               Walmart Sales                           

In [ ]:
!kaggle datasets download -d chetankv/dogs-cats-images

 98% 427M/435M [00:03<00:00, 92.6MB/s]
100% 435M/435M [00:03<00:00, 129MB/s] 


In [ ]:
!unzip -q dogs-cats-images.zip

In [ ]:
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
import numpy as np
import os
import pickle

# Initialize VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(50, 50, 3))
model = Model(inputs=base_model.input, outputs=base_model.output)

def extract_features(directory, model, limit=None):
    features, labels = [], []
    categories = ['cats', 'dogs']  # Assuming these are your folder names
    for cat in categories:
        path = os.path.join(directory, cat)
        images = os.listdir(path)[:limit]  # Limit the number of images processed
        for img_name in images:
            img_path = os.path.join(path, img_name)
            img = image.load_img(img_path, target_size=(50, 50))
            img_array = image.img_to_array(img)
            img_array_expanded_dims = np.expand_dims(img_array, axis=0)
            preprocessed_img = preprocess_input(img_array_expanded_dims)
            features.append(model.predict(preprocessed_img)[0].flatten())
            labels.append(categories.index(cat))
    return np.array(features), np.array(labels)

# Extract features for training data
train_dir = "dataset/training_set"  # Update this path
X, y = extract_features(train_dir, model, limit=1680)  # Limit to 300 images per category

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define and train SVM with GridSearchCV
param_grid = {'C': [ 1], 'gamma': ['scale', 'auto'], 'kernel': ['linear', 'rbf']}
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2, cv=3)
grid.fit(X_train, y_train)

# Save the best SVM model
with open('optimized_svc_model.pkl', 'wb') as file:
    pickle.dump(grid.best_estimator_, file)

1/1 [==============================] - 0s 54ms/step
Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] END ....................C=1, gamma=scale, kernel=linear; total time= 7.7min
[CV] END ....................C=1, gamma=scale, kernel=linear; total time=  15.8s


In [ ]:
import numpy as np
import pickle
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model

# Function to preprocess and extract features from a single image
def preprocess_and_extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(50, 50))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(img_array_expanded_dims)
    features = model.predict(preprocessed_img)
    return features.flatten().reshape(1, -1)

# Load the trained and optimized SVM model
with open('optimized_svc_model.pkl', 'rb') as file:
    loaded_svc = pickle.load(file)

# Load the VGG16 model for feature extraction
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(50, 50, 3))
feature_model = Model(inputs=base_model.input, outputs=base_model.output)

# Specify the path to your image
image_path = 'dataset/test_set/dogs/namdb.png'  # Update this path

# Extract features and predict
features = preprocess_and_extract_features(image_path, feature_model)
predicted_label = loaded_svc.predict(features)

# Translate label to category
categories = ['Cat', 'Dog']
predicted_category = categories[int(predicted_label[0])]
print(f'The image is predicted to be a: {predicted_category}')

1/1 [==============================] - 0s 486ms/step
The image is predicted to be a: Dog


In [ ]:
import numpy as np
import pickle
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from sklearn.metrics import accuracy_score, classification_report
import os

# Function to preprocess and extract features from images in a directory
def preprocess_and_extract_features_directory(directory, model, categories, limit=None):
    features = []
    labels = []
    for category in categories:
        path = os.path.join(directory, category)
        print(f"Processing category: {category}, Path: {path}")  # Debugging print
        if not os.path.isdir(path):
            print(f"Directory does not exist: {path}")  # Debugging print
            continue
        img_files = os.listdir(path)[:limit]
        for img_file in img_files:
            img_path = os.path.join(path, img_file)
            print(f"Processing image: {img_path}")  # Debugging print
            try:
                img_feature = preprocess_and_extract_features(img_path, model)
                features.append(img_feature)
                labels.append(categories.index(category))
            except Exception as e:
                print(f"Failed to process {img_path}: {e}")  # Debugging print
    if len(features) == 0:
        raise ValueError("No features were extracted. Check the dataset path and image processing.")
    return np.vstack(features), np.array(labels)


# Assuming the directory structure and file paths are set up correctly
test_dir = "dataset/test_set"  # Update this path to your test dataset directory
categories = ['cats', 'dogs']  # Ensure this matches the directory names in your test set

# Extract features and labels from the test dataset
X_test, y_test = preprocess_and_extract_features_directory(test_dir, feature_model, categories)

# Predict the labels of the test data using the loaded SVM model
predicted_labels = loaded_svc.predict(X_test)

# Calculate and print the accuracy
test_accuracy = accuracy_score(y_test, predicted_labels)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Optionally, print a detailed classification report
print(classification_report(y_test, predicted_labels, target_names=categories))

Processing category: cats, Path: dataset/test_set/cats
Processing image: dataset/test_set/cats/cat.4611.jpg
1/1 [==============================] - 0s 59ms/step
Processing image: dataset/test_set/cats/cat.4301.jpg
1/1 [==============================] - 0s 64ms/step
Processing image: dataset/test_set/cats/cat.4623.jpg
1/1 [==============================] - 0s 61ms/step
Processing image: dataset/test_set/cats/cat.4462.jpg
1/1 [==============================] - 0s 59ms/step
Processing image: dataset/test_set/cats/cat.4423.jpg
1/1 [==============================] - 0s 74ms/step
Processing image: dataset/test_set/cats/cat.4465.jpg
1/1 [==============================] - 0s 60ms/step
Processing image: dataset/test_set/cats/cat.4267.jpg
1/1 [==============================] - 0s 58ms/step
Processing image: dataset/test_set/cats/cat.4850.jpg
1/1 [==============================] - 0s 61ms/step
Processing image: dataset/test_set/cats/cat.4948.jpg
1/1 [==============================] - 0s 59ms/step
